# Discourse Context Frequency - Historical Sociolinguistics

Paste abstract here.

[https://www.gutenberg.org/cache/epub/1619/pg1619.txt](https://www.gutenberg.org/cache/epub/1619/pg1619.txt)

In [1]:
text = []
with open('data/celestina.txt') as f:
    text = f.read()

In [2]:
import re

Remove unnecessary text at the start. The text begins with the title `[LA CELESTINA]`.

In [3]:
re.search("\[LA CELESTINA\]", text)

<re.Match object; span=(12671, 12685), match='[LA CELESTINA]'>

In [4]:
text_start = text[12671:]
len(text_start)

656375

Remove unnecessary text at the end. The final line of the text is `[A Dios gracias.]`.

In [5]:
re.search("\[A Dios gracias.\]", text_start)

<re.Match object; span=(381463, 381480), match='[A Dios gracias.]'>

In [6]:
new_text = text_start[:381481]
len(new_text)

381481

The text contains many linebreaks `\n`, sometimes multiple in a row. Before beginning, we will replace all instances of `\n` with a single space.

In [7]:
text_2 = re.sub("(\\n)+", " ", new_text)

Remove instances of double spaces.

In [8]:
text_3 = re.sub("\s\s", " ", text_2)

## Identify token words

Move token words into dataframe of structure:

- token
- token frequency in text (log frequency per million)
- preceeding phonological context
- following phonological context
- transmission history (oral, written)
- favorable preceeding (yes, no)
- favorable following (yes, no)
- lexical stress (stress, unstress)
- FFC (proportion of tokens found in favorable)
- obstruent in original Latin (p, f, k)
- hapax token (yes if only instance, no if multiple instances)
- Modern Spanish realization (preservation, palatalization)

The following regex expression will find all words beginning with cl-, fl-, pl- or ll- and will give the surrounding context for each instance.

In [10]:
words = re.findall(r'.{0,20}\s[p,f,c,l]l\w+.{0,2}', text_3)

In [11]:
len(words)

516

In [12]:
len(set(words))

516

In [13]:
context_ind = [m.span() for m in re.finditer(r'.{0,20}\s[p,f,c,l]l\w+.{0,2}', text_3)]
len(context_ind)

516

In [14]:
context = []
for x in context_ind:
    start = x[0]
    new_start = start - 10
    end = x[1]
    new_end = end + 25
    
    context.append(text_3[new_start:new_end])
len(context)    

516

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(words)
df = df.rename(columns={0: "context_small"})
df.head()

,context_small
0,de Milan: mas en los claros i
1,tificio: su fuerte e claro m
2,las de su perdicion: lleuaronla e
3,"ue ella, por ceuo la lleuan:"
4,on es que aplique mi pluma e


In [17]:
df["context_large"] = context

In [18]:
df["word"] = df["context_small"].apply(lambda x: re.search(r'\s[pflc]l\w+', x).group())

In [19]:
df["word"] = df["word"].apply(lambda x: re.sub(r'\s', "", x))
df.head()

,context_small,context_large,word
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma


In [20]:
df["prev"] = df["context_large"].apply(lambda x: re.search(r'.\s[pflc]l', x).group())
df["prev"] = df["prev"].apply(lambda x: re.sub(r'(.)\s[pflc]l', (r"\1"), x))
df.head()

,context_small,context_large,word,prev
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i


In [21]:
import numpy as np

In [22]:
df["prev_fav"] = np.where((df['prev'] == "e") | (df['prev'] == "a") | (df['prev'] == "o"), "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no


In [24]:
df["fol"] = df["context_large"].apply(lambda x: re.search(r'.\s[pflc]l\w', x).group())
df["fol"] = df["fol"].apply(lambda x: re.sub(r'.\s[pflc]l(\w)', (r"\1"), x))
df.head()

,context_small,context_large,word,prev,prev_fav,fol
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u


In [25]:
df["fol_fav"] = np.where((df['fol'] == "e") | (df['fol'] == "a") | (df['fol'] == "o"), "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no


## Find token frequency (ppm) for each token word

In [26]:
import nltk
from nltk.tokenize import RegexpTokenizer

In [27]:
tokenizer = RegexpTokenizer(r'\w+')
total_words = tokenizer.tokenize(text_3)
total_words = len(total_words)

In [28]:
df["count"] = df['word'].apply(lambda x: len(re.findall(x, text_3)))
df["frequency_ppm"] = df["count"].apply(lambda x: (x/total_words)*1000000)
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031


In [30]:
import math
df["freq_log_ppm"] = df["frequency_ppm"].apply(lambda x: math.log(x))
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm,freq_log_ppm
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322,4.474040
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901,5.817774
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774,3.375427
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548,4.068574
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031,5.247229


## Hapax tokens

In [31]:
df["hapax"] = np.where(df["count"] == 1, "yes", "no")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm,freq_log_ppm,hapax
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322,4.474040,no
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901,5.817774,no
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774,3.375427,no
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548,4.068574,no
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031,5.247229,no


## Lexical Stress

In [32]:
from syltippy import syllabize

In [33]:
df["stress"] = np.where(syllabize(df["word"])[1]==0, "stressed", "unstressed")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm,freq_log_ppm,hapax,stress
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322,4.474040,no,stressed
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901,5.817774,no,stressed
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774,3.375427,no,stressed
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548,4.068574,no,stressed
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031,5.247229,no,stressed


## FFC: Frequency in favorable environment

In [40]:
values = df.groupby("word")["prev_fav"].value_counts()
values_df = values.to_frame()
values_df = values_df.rename(columns={"prev_fav": "fav_count"})
values_df.head()

fav_count
word     prev_fav           
clamor   yes               1
clara    no                4
         yes               3
claras   no                3
claridad yes               4

In [41]:
values_df = values_df.reset_index()
values_df.head()

,word,prev_fav,fav_count
0,clamor,yes,1
1,clara,no,4
2,clara,yes,3
3,claras,no,3
4,claridad,yes,4


In [42]:
values_df_fav = values_df[values_df["prev_fav"]=="yes"].copy()
values_df_fav.head()

,word,prev_fav,fav_count
0,clamor,yes,1
2,clara,yes,3
4,claridad,yes,4
7,claro,yes,9
11,claros,yes,1


In [47]:
values_df_fav = values_df_fav.drop(['prev_fav'], axis = 1)
values_df_fav.head()

,word,fav_count
0,clamor,1
2,clara,3
4,claridad,4
7,claro,9
11,claros,1


In [34]:
len(df)

516

In [48]:
df = df.merge(values_df_fav, how="left", on="word")
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm,freq_log_ppm,hapax,stress,fav_count
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322,4.474040,no,stressed,1.0
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901,5.817774,no,stressed,9.0
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774,3.375427,no,stressed,NaN
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548,4.068574,no,stressed,4.0
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031,5.247229,no,stressed,2.0


In [49]:
df['fav_count'] = df['fav_count'].fillna(0)

In [53]:
df['FFC'] = df['fav_count']/df['count']
df.head()

,context_small,context_large,word,prev,prev_fav,fol,fol_fav,count,frequency_ppm,freq_log_ppm,hapax,stress,fav_count,FFC
0,de Milan: mas en los claros i,herrerias de Milan: mas en los claros ingenios...,claros,s,no,a,yes,6,87.710322,4.474040,no,stressed,1.0,0.166667
1,tificio: su fuerte e claro m,: sotil artificio: su fuerte e claro metal: su...,claro,e,yes,a,yes,23,336.222901,5.817774,no,stressed,9.0,0.391304
2,las de su perdicion: lleuaronla e,tose con alas de su perdicion: lleuaronla en a...,lleuaronla,:,no,e,yes,2,29.236774,3.375427,no,stressed,0.0,0.000000
3,"ue ella, por ceuo la lleuan:","rtes mas que ella, por ceuo la lleuan: en las ...",lleuan,a,yes,e,yes,4,58.473548,4.068574,no,stressed,4.0,1.000000
4,on es que aplique mi pluma e,dano: razon es que aplique mi pluma este enga...,pluma,i,no,u,no,13,190.039031,5.247229,no,stressed,2.0,0.153846


In [54]:
df.to_csv("data/partial.csv", index=False)